In [10]:
"""Data processed for CNN and SVM models"""

import os
import pandas as pd
import numpy as np
from PIL import Image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Set data directory path
data_dir = '../data/raw/'
images_dir = os.path.join(data_dir, 'images')
print(images_dir)
labels_path = os.path.join(data_dir, 'labels_agu.csv')

# Read label data
labels_df = pd.read_csv(labels_path, names=['image_id', 'probability', 'type'], sep='\s+')
# 映射概率到类别
def map_probability_to_class(probability):
    if probability == 0.0:
        return 0
    elif 0.33 <= probability < 0.4:
        return 1
    elif 0.66 <= probability < 0.68:
        return 2
    elif probability == 1.0:
        return 3
    else:
        raise ValueError(f"未知的概率值: {probability}")

labels_df['class'] = labels_df['probability'].apply(map_probability_to_class)

def load_images(image_paths, image_dir):
    images = []
    for image_path in image_paths:
        full_path = os.path.join(image_dir, image_path)
        with Image.open(full_path) as img:
            img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
            if img_array.ndim == 2:
                # If the image is grayscale, convert it to a three-channel grayscale image
                img_array = np.stack((img_array,)*3, axis=-1)
            images.append(img_array)
    return np.array(images)


image_paths = labels_df['image_id'].tolist()
images = load_images(image_paths, data_dir)

# Ensure images have one channel
if images.ndim == 3:
    images = np.expand_dims(images, axis=-1)

# Convert class labels to one-hot encoding
labels = to_categorical(labels_df['class'].values, num_classes=4)

X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.25, stratify=labels, random_state=42,shuffle=True
)


# Save split dataset to files
np.save('../data/processed/X_train_cs.npy', X_train)
np.save('../data/processed/X_test_cs.npy', X_test)
np.save('../data/processed/y_train_cs.npy', y_train)
np.save('../data/processed/y_test_cs.npy', y_test)

# Confirm saved data
print(f"Training set images shape: {X_train.shape}")
print(f"Test set images shape: {X_test.shape}")
print(f"Training set labels shape: {y_train.shape}")
print(f"Test set labels shape: {y_test.shape}")
# Apply stratified sampling to ensure consistency in class distribution between training set and test set

../data/raw/images
Training set images shape: (2267, 300, 300, 3)
Test set images shape: (756, 300, 300, 3)
Training set labels shape: (2267, 4)
Test set labels shape: (756, 4)


In [20]:
"""Data_types processed for CNN and SVM models"""
import os
import pandas as pd
import numpy as np
from PIL import Image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
import cv2
from skimage import filters


# Path setup
data_dir = '../data/raw'
labels_file = '../data/raw/labels_agu.csv'

# Load label file and specify column names
labels_df = pd.read_csv(labels_file, names=['image_id', 'probability', 'type'], sep='\s+')
labels_df['class'] = labels_df['probability'].apply(map_probability_to_class)

# Map probabilities to classes
def map_probability_to_class(probability):
    # Map probabilities to classes
    if probability == 0.0:
        return 0
    elif probability == 0.33:
        return 1
    elif probability == 0.67:
        return 2
    elif probability == 1.0:
        return 3
    else:
        raise ValueError(f"Unknown probability value: {probability}")



def convert_color_space(image, target_space):
    # Convert color space of the image
    if target_space == 'HSV':
        converted_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    elif target_space == 'LAB':
        converted_image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    else:
        raise ValueError(f"Unknown target_space: {target_space}")
    return converted_image

def edge_enhancement(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_image = np.uint8(gray_image)
    # Edge detection using Canny algorithm
    edges = cv2.Canny(gray_image, 300, 300)
    # Convert single channel edge image to three channels
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    # Convert edge image to same data type as original image (if needed)
    if edges_colored.dtype != image.dtype:
        edges_colored = edges_colored.astype(image.dtype)

    # Blend images
    enhanced_image = cv2.addWeighted(image, 0.8, edges_colored, 0.2, 0)
    
    return enhanced_image

def load_images(image_paths, image_dir):
    images = []
    for image_path in image_paths:
        full_path = os.path.join(image_dir, image_path)
        with Image.open(full_path) as img:
            img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
            if img_array.ndim == 2:
                # If the image is grayscale, convert it to a three-channel grayscale image
                img_array = np.stack((img_array,)*3, axis=-1)
            images.append(img_array)
    return np.array(images)

def preprocess_image_for_transformer(image_path, image_type, target_size=(300, 300)):
    # Load image file
    image = load_img(image_path, target_size=target_size)
    # Convert it to array
    image = img_to_array(image)
    # Apply different preprocessing based on image type
    if image_type == 0 :  # Assume 0 represents "mono"
        # Apply specific preprocessing for "mono" type images
        image = convert_color_space(image, 'HSV')
        # Enhance edges using Sobel operator
        image = edge_enhancement(image)
        # Contrast enhancement - CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        for i in range(image.shape[-1]):
            image[..., i] = clahe.apply(image[..., i].astype('uint8'))
        # Noise reduction - Gaussian Blur
        image = cv2.GaussianBlur(image, (5, 5), 0)
        # pass
        
    elif image_type == 0:  # Assume 1 represents "poly"
        # pass
        # Apply specific preprocessing for "poly" type images
        image = convert_color_space(image, 'LAB')
        # Edge detection in x direction
        sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)
        # Edge detection in y direction
        sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
        # Compute absolute values
        abs_sobelx = np.absolute(sobelx)
        abs_sobely = np.absolute(sobely)
        # Combine Sobel x and y
        sobel_combined = np.sqrt(np.square(abs_sobelx) + np.square(abs_sobely))
        sobel_combined = np.clip(sobel_combined, 0, 255).astype('uint8')
        image = sobel_combined
        # Contrast enhancement - CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        for i in range(image.shape[-1]):
            image[..., i] = clahe.apply(image[..., i].astype('uint8'))

    image = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
    # Normalize pixel values
    return image


# image_paths = labels_df['image_id'].tolist()
# images = load_images(image_paths, data_dir)
image_paths = labels_df['image_id'].apply(lambda x: os.path.join(data_dir, x))

# Ensure images have one channel
if images.ndim == 3:
    images = np.expand_dims(images, axis=-1)

# Initialize LabelEncoder
type_label_encoder = LabelEncoder()

# Convert type labels to integer form
types_encoded = type_label_encoder.fit_transform(labels_df['type'].values)
# Now types_encoded contains integer labels for type information, which can be used as one of the inputs for the model

# Convert type labels to integers
types_encoded = types_encoded.reshape(-1, 1)  # Reshape to (num_samples, 1)

# Use a list comprehension to preprocess all images and include type information
X = np.array([preprocess_image_for_transformer(path, types_encoded[idx][0]) for idx, path in enumerate(image_paths)])

y_one_hot = to_categorical(labels_df['class'].values, num_classes=4)

# Stratify split dataset by types
X_train, X_test, y_train, y_test, types_train, types_test = train_test_split(
    X, y_one_hot, types_encoded, test_size=0.25, stratify=types_encoded, random_state=42,shuffle=True
)


# 保存分割后的数据集到文件
np.save('../data/processed/X_train_cs_type_pc.npy', X_train)
np.save('../data/processed/X_test_cs_type_pc.npy', X_test)
np.save('../data/processed/y_train_cs_type_pc.npy', y_train)
np.save('../data/processed/y_test_cs_type_pc.npy', y_test)
np.save('../data/processed/types_train_cs_type_pc.npy', types_train)
np.save('../data/processed/types_test_cs_type_pc.npy', types_test)

# 确认已保存的数据
print(f"Training set images shape: {X_train.shape}")
print(f"Test set images shape: {X_test.shape}")
print(f"Training set labels shape: {y_train.shape}")
print(f"Test set labels shape: {y_test.shape}")
print(f"Training set types shape: {types_train.shape}")
print(f"Test set types shape: {types_test.shape}")



Training set images shape: (2267, 300, 300, 3)
Test set images shape: (756, 300, 300, 3)
Training set labels shape: (2267, 4)
Test set labels shape: (756, 4)
Training set types shape: (2267, 1)
Test set types shape: (756, 1)


In [ ]:
"""Normal transformer data processed"""

import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
from IPython.display import display

def preprocess_image_for_transformer(image_path, size=(300, 300), num_patches=16):
   # Load image and resize
   image = load_img(image_path, target_size=size, color_mode='rgb')
   image = img_to_array(image)

   # Add other necessary image preprocessing steps here
   # Contrast enhancement - CLAHE
   clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
   for i in range(image.shape[-1]):
       image[..., i] = clahe.apply(image[..., i].astype('uint8'))
   # Noise reduction - Gaussian blur
   image = cv2.GaussianBlur(image, (5, 5), 0)
   # Normalize pixel values
   image = image / 255.0
   return image

# Set data directory path and labels file
data_dir = '../data/raw'
labels_file = '../data/raw/labels.csv'

# Read labels file and check column names
labels_df = pd.read_csv(labels_file)
print(labels_df.head())

# Read labels file and specify column names
labels_df = pd.read_csv(labels_file, names=['image_id', 'probability', 'type'], sep='\s+')

image_paths = labels_df['image_id'].apply(lambda x: os.path.join(data_dir, x))
labels = labels_df['probability'].values

# Preprocess images
X = np.array([preprocess_image_for_transformer(path) for path in image_paths])
y = np.array(labels)

# Convert labels to one-hot encoding
y = to_categorical(y, num_classes=4)
print(y[:50])

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.25, random_state=42, stratify=y
)
print(y_test.shape)

# Save preprocessed data to files
np.save('../data/processed/X_train_transformer.npy', X_train)
np.save('../data/processed/X_test_transformer.npy', X_test)
np.save('../data/processed/y_train_transformer.npy', y_train)
np.save('../data/processed/y_test_transformer.npy', y_test)

In [2]:
# Save split dataset_types to file
np.save('../data/processed/X_train_transformer_types.npy', X_train)
np.save('../data/processed/X_test_transformer_types.npy', X_test)
np.save('../data/processed/y_train_transformer_types.npy', y_train)
np.save('../data/processed/y_test_transformer_types.npy', y_test)
np.save('../data/processed/types_train.npy', types_train)
np.save('../data/processed/types_test.npy', types_test)

In [1]:
"""Data_augmention types classify model transformer
"""
import os
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
import cv2
from skimage import filters
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def convert_color_space(image, target_space):
    # Convert color space of the image
    if target_space == 'HSV':
        converted_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    elif target_space == 'LAB':
        converted_image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    else:
        raise ValueError(f"Unknown target_space: {target_space}")
    return converted_image

def edge_enhancement(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray_image = np.uint8(gray_image)
    # Edge detection using Canny algorithm
    edges = cv2.Canny(gray_image, 300, 300)
    # Convert single channel edge image to three channels
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    # Convert edge image to same data type as original image (if needed)
    if edges_colored.dtype != image.dtype:
        edges_colored = edges_colored.astype(image.dtype)

    # Blend images
    enhanced_image = cv2.addWeighted(image, 0.8, edges_colored, 0.2, 0)
    
    return enhanced_image


def preprocess_image_for_transformer(image_path, image_type, target_size=(300, 300)):
    # Load image file
    image = load_img(image_path, target_size=target_size)
    # Convert it to array
    image = img_to_array(image)
    # Apply different preprocessing based on image type
    if image_type == 0 :  # Assume 0 represents "mono"
        # Apply specific preprocessing for "mono" type images
        image = convert_color_space(image, 'HSV')
        # Enhance edges using Sobel operator
        image = edge_enhancement(image)
        # Contrast enhancement - CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        for i in range(image.shape[-1]):
            image[..., i] = clahe.apply(image[..., i].astype('uint8'))
        # Noise reduction - Gaussian Blur
        image = cv2.GaussianBlur(image, (5, 5), 0)
        
    elif image_type == 0:  # Assume 1 represents "poly"
        # Apply specific preprocessing for "poly" type images
        image = convert_color_space(image, 'LAB')
        # Edge detection in x direction
        sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)
        # Edge detection in y direction
        sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
        # Compute absolute values
        abs_sobelx = np.absolute(sobelx)
        abs_sobely = np.absolute(sobely)
        # Combine Sobel x and y
        sobel_combined = np.sqrt(np.square(abs_sobelx) + np.square(abs_sobely))
        sobel_combined = np.clip(sobel_combined, 0, 255).astype('uint8')
        image = sobel_combined
        # Contrast enhancement - CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        for i in range(image.shape[-1]):
            image[..., i] = clahe.apply(image[..., i].astype('uint8'))

    # Preprocess image for EfficientNetB0
    image = preprocess_input(image)
    # Normalize pixel values
    return image


# Map probabilities to classes
def map_probability_to_class(probability):
    # Map probabilities to classes
    if probability == 0.0:
        return 0
    elif probability == 0.33:
        return 1
    elif probability == 0.67:
        return 2
    elif probability == 1.0:
        return 3
    else:
        raise ValueError(f"Unknown probability value: {probability}")
    

# Path setup
data_dir = '../data/raw'
labels_file = '../data/raw/labels_agu.csv'

# Load label file and specify column names
labels_df = pd.read_csv(labels_file, names=['image_id', 'probability', 'type'], sep='\s+')
image_paths = labels_df['image_id'].apply(lambda x: os.path.join(data_dir, x))

labels_df['class'] = labels_df['probability'].apply(map_probability_to_class)

# Initialize LabelEncoder
type_label_encoder = LabelEncoder()

# Convert type labels to integer form
types_encoded = type_label_encoder.fit_transform(labels_df['type'].values)
# Now types_encoded contains integer labels for type information, which can be used as one of the inputs for the model

# Convert type labels to integers
types_encoded = types_encoded.reshape(-1, 1)  # Reshape to (num_samples, 1)

# Use a list comprehension to preprocess all images and include type information
X = np.array([preprocess_image_for_transformer(path, types_encoded[idx][0]) for idx, path in enumerate(image_paths)])

# Convert class labels to one-hot encoding
y_one_hot = to_categorical(labels_df['class'].values, num_classes=4)

# Stratify split dataset by types
X_train, X_test, y_train, y_test, types_train, types_test = train_test_split(
    X, y_one_hot, types_encoded, test_size=0.25, stratify=types_encoded, random_state=42,shuffle=True
)


# Save split dataset_types to file
np.save('../data/processed/X_train_transformer_types_aug.npy', X_train)
np.save('../data/processed/X_test_transformer_types_aug.npy', X_test)
np.save('../data/processed/y_train_transformer_types_aug.npy', y_train)
np.save('../data/processed/y_test_transformer_types_aug.npy', y_test)
np.save('../data/processed/types_train_aug.npy', types_train)
np.save('../data/processed/types_test_aug.npy', types_test)


In [18]:
import os
import numpy as np
# Set the path to the dataset
base_path = os.path.dirname(os.getcwd()) 
model_path = os.path.join(base_path, 'models/transformer_with_type_aug_model.h5') 

X_test = np.load('../data/processed/X_test_transformer_types_aug.npy')
y_test = np.load('../data/processed/y_test_transformer_types_aug.npy')
types_test = np.load('../data/processed/types_test_aug.npy')

print(f"Training set images shape: {X_train.shape}")
print(f"Test set images shape: {X_test.shape}")
print(f"Training set labels shape: {y_train.shape}")
print(f"Test set labels shape: {y_test.shape}")
print(f"Training set types shape: {types_train.shape}")
print(f"Test set types shape: {types_test.shape}")


Training set images shape: (2267, 300, 300, 3)
Test set images shape: (756, 300, 300, 3)
Training set labels shape: (2267, 4)
Test set labels shape: (756, 4)
Training set types shape: (2267, 1)
Test set types shape: (756, 1)
